### Emma Klemets, 260775167   
All my work, code, output and written answers are in this notebook.

In [ ]:
import numpy as np
import matplotlib

#for pop out plots - windows
matplotlib.use('Qt5Agg')
import matplotlib.pyplot as plt
import time
import matplotlib.cm as cm
from mpl_toolkits.mplot3d import Axes3D

from particleClass import *

Make a 3-D nbody code that calculates the forces by computing the potential, where the potential is found by convolving the density with the (softened) potential from a single particle. The acceleration is then found by taking the gradient of the potential. You will probably wish to use a leapfrog solver with fixed timestep.

In [ ]:
"""
calculates the forces by computing the potential

potential is found by convolving the density with the (softened) potential from a single particle. 
phi = rho x V_1

The acceleration is then found by taking the gradient of the potential.
acc = grad phi


calculate the potential using the Green's function!
So you can use the Green's function of the laplacian : -1/(4*pi*r) where r = sqrt(x^2 + y^2 + z^2)
Since (taking 4*pi*G=1): laplacian(phi) = rho ,

solve phi = ifft(fft(G_laplacian) * fft(rho))  where G_laplacian is just the Green's function of the laplacian

- how does it not matter which particle for rho?
- what is the r in G's relative to - should it be between two particles? 
"""
"""
To do later:
- add in G value for gravity (right now just 1 I guess)

With non-p I basicly have no particles by the end - make grid bigger, start particles more in center
still not working, should collapse in center, but instead they all just run away
"""
x=1

In [ ]:
n=10000 #runs really quite slow with 100 000 points
gridSize = [20, 20, 20]

#single particle at rest
x=np.random.uniform(0, gridSize[0],(n,3))
v=np.random.randn(n,3)*0 #at rest?

# print(x)

m=np.ones(n) #mass

dt_t = 1.

# parts=Particles(x,v,m, gridSize, grid_dx=dt_t)
parts=Particles(x,v,m, gridSize, grid_dx=dt_t, BC_per=False)

dt=0.01

# x_g = np.arange(0, gridSize[0], dt_t)
# y_g = np.arange(0, gridSize[1], dt_t)
# z_g = np.arange(0, gridSize[2], dt_t)
# grid = np.meshgrid(x_g, y_g, z_g)
# xx,yy,zz = np.meshgrid(x_g, y_g, z_g)

phi_test, f_grid, grad_x, grad_y, grad_z = parts.get_forces_conv()

d = parts.get_density()
# print(phi_test[0].shape)

# print(yy.shape)
# print(phi_test[0])

# """
x=np.arange(0, gridSize[0], dt_t)
yy,zz=np.meshgrid(x,x)
"""
for i in range(0, int(gridSize[0]/dt_t)):
    plt.pcolor(yy, zz, grad_x[i],  shading='auto')
#     plt.pcolor(yy,zz, g[i],  shading='auto')
    
    plt.title("x = {}".format(i*dt_t))
    plt.xlabel("z")
    plt.ylabel("y")
    plt.colorbar()
    plt.pause(2)
    plt.clf()
"""


# https://colab.research.google.com/drive/1zc1nvra1HF01mTtpOlRigj-UBVzYgFqq#scrollTo=pvtJPFS3LfYv

fig = plt.figure()
ax = Axes3D(fig)
ax.set_xlabel("x",fontsize=14)
ax.set_ylabel("y",fontsize=14)
ax.set_zlabel("z",fontsize=14)

# """
ax.set_xlim3d(0, gridSize[0])
ax.set_ylim3d(0,gridSize[1])
ax.set_zlim3d(0,gridSize[2])

ax.set_title("Initial conditions of the Universe\n",fontsize=20)
scatt = ax.scatter(parts.x[:,0], parts.x[:,1], parts.x[:,2], color="royalblue",marker=".")#,s=.1)

plt.pause(1.)

save_E = []

# """   

for k in range(50):
    for l in range(10):
        parts.take_step_leapfrog_conv(dt)
   
    scatt.remove() #clears prev points
    scatt = ax.scatter(parts.x[:,0], parts.x[:,1], parts.x[:,2], color="royalblue",marker=".", alpha=0.7)
#     ax.set_title("frame {}".format(k))
#     ax.text2D(0.05, 0.95, "frame {}".format(k), transform=ax.transAxes)

    ax.set_title("frame {}".format(k),fontsize=20)
    plt.savefig('nonP_{}.jpg'.format(k), dpi=1200)

    save_E.append(parts.get_energy())
    print("E total:", save_E[-1][0])
    
#     i = 1
#     phi = parts.potential
#     plt.pcolor(yy, zz, phi[i],  shading='auto')
    
#     plt.title("x = {}".format(i*dt_t))
#     plt.xlabel("z")
#     plt.ylabel("y")
#     plt.colorbar()
#     plt.pause(2)
#     plt.clf()
    
#     plt.pause(0.1)
    
# """   

In [ ]:
# print(save_E[:,0])

In [ ]:
save_E = np.array(save_E)
steps = np.linspace(0, len(save_E[:,0])-1, len(save_E[:,0]))

fig = plt.figure()
# plt.plot(steps, save_E[:,0], label="Total E")
# plt.plot(steps, save_E[:,1], label="Kinetic E", ls="--") 
plt.plot(steps, save_E[:,2], label="Potential E") #hmm this is constant - should it be?

plt.title("Energy over time")
plt.xlabel("time step")
plt.ylabel("Energy")
plt.legend()

plt.pause(5.)

### Part 1: 
*Using this code, show that a single particle starting at rest remains motionless.*

In [ ]:
n=1 #single particle

#set size of grid and size of cells
gridSize = [5, 5, 5]
dt_t = 0.2

#set up our single particle at rest, position chosen pseudo-randomly 
x=np.random.uniform(0.5, gridSize[0]-0.5,(n,3))
v=np.random.randn(n,3)*0 #at rest

#mass of 1
m=np.ones(n) #mass

#initialize the particle object
parts=Particles(x,v,m, gridSize, grid_dx=dt_t)

#time step
dt=0.01

#colors for plotting
colors = cm.rainbow(np.linspace(0, 1, n))

#set up 3D figure
fig = plt.figure()
ax=fig.add_subplot(111,projection="3d")

ax.set_xlabel("x",fontsize=14)
ax.set_ylabel("y",fontsize=14)
ax.set_zlabel("z",fontsize=14)

ax.set_xlim3d(0, gridSize[0])
ax.set_ylim3d(0,gridSize[1])
ax.set_zlim3d(0,gridSize[2])

#loop to take time steps
for k in range(100):
    for l in range(10):
        parts.take_step_leapfrog_conv(dt)
        
    #only saving a data point every 10 steps to show more time lapsed
    for n_i in range(n):
        scatt = ax.scatter(parts.x[n_i][0], parts.x[n_i][1], parts.x[n_i][2], color=colors[n_i])

    #prints out current step number and position - could also add actual time here
    print("Step: {}, E total: {}".format(k,  parts.get_energy()[0]))
    
    #saving frames to make a gif out of them after
    ax.set_title("frame {}".format(k),fontsize=20)
    plt.savefig('SinglePart_{}.jpg'.format(k), dpi=1200)
    scatt.remove() #clears prev points


### Part 2: 
Next, show that a pair of particles placed in a circular orbit continue to orbit each other, for at least some reasonable length of time. 

In [ ]:
n=2
#two particles in circular orbit

gridSize = [10, 10, 10]

x=np.zeros([n,3])
v=np.zeros([n,3])
x[0,0]=gridSize[0]/2
x[0,1]=gridSize[1]/2
x[0,2]=gridSize[2]/2 - 1

x[1,0]=gridSize[0]/2
x[1,1]=gridSize[1]/2
x[1,2]=gridSize[2]/2 + 1

# v[0,1]=0.25
# v[1,1]=-0.25
# v[0,2]=0.25
# v[1,2]=-0.25

v[0,0]=0.1
v[0,1]=0.1

v[1,0]=-.1
v[1,1]=-.1

m=np.ones(n)*10 #mass

dt_t = 0.5#1.0
parts=Particles(x,v,m, gridSize, grid_dx=dt_t)

dt=0.05

colors = cm.rainbow(np.linspace(0, 1, n)) 

fig = plt.figure()
ax = Axes3D(fig)
        
for k in range(100):
    for l in range(1):
        parts.take_step_leapfrog_conv(dt)
    
#     print('step ',k)

    #     plt.clf() #clears prev graphs
    for n_i in range(n):
        if k==0:
            ax.scatter(parts.x[n_i][0], parts.x[n_i][1], parts.x[n_i][2])#, color=colors[n_i])
            
        else:
            ax.scatter(parts.x[n_i][0], parts.x[n_i][1], parts.x[n_i][2], color=colors[n_i])
    
#     ax.set_title("frame {}".format(k),fontsize=20)
#     plt.savefig('3dfinal_wrap_{}.jpg'.format(k), dpi=1200)
    plt.pause(0.01)

In [ ]:
"""
Makes a cool pattern, but not a circle
x[0,0]=2.3 #p1 x
x[0,1]=2.5 #p1 y
x[0,2]=2.5 #p1 z

x[1,0]=2.8 #p2 x
x[1,1]=2.5 #p2 y
x[1,2]=2.5 #p2 z

v[0,0]=0.3
v[0,1]=0.3

v[1,0]=-0.3
v[1,1]=-0.3

gridSize = np.array([5, 5, 5])

#     positions = np.array([[0.5,0.5,0.5],[0.5,0.6,0.6]])
#     mass = np.ones(2)*5
#     vel = np.array([[0.04,0.04,0], [-0.04,-0.04, 0]])

m=np.ones(n)*8 #mass

dt_t = .5#1.0"""

"""
This is very, very close to a circle - without green flip thing
x=np.zeros([n,3])
v=np.zeros([n,3])
x[0,0]=2.4 #p1 x
x[0,1]=2.5 #p1 y
x[0,2]=2.5 #p1 z

x[1,0]=2.6 #p2 x
x[1,1]=2.5 #p2 y
x[1,2]=2.5 #p2 z

# vel = np.sqrt(5/(2*np.abs(x[0,0]-x[1,0])))
vel = 0.3


v[0,0]=0
v[0,1]=vel

v[1,0]=0
v[1,1]=-vel

gridSize = np.array([5, 5, 5])

m=np.ones(n)*5 #mass

dt_t = 0.5#1.0
parts=Particles(x,v,m, gridSize, grid_dx=dt_t)
colors = cm.rainbow(np.linspace(0, 1, n))

dt=0.01
"""

In [23]:
n=2

x=np.zeros([n,3])
v=np.zeros([n,3])
x[0,0]=2.4 #p1 x
x[0,1]=2.5 #p1 y
x[0,2]=2.5 #p1 z

x[1,0]=2.6 #p2 x
x[1,1]=2.5 #p2 y
x[1,2]=2.5 #p2 z

# vel = np.sqrt(5/(2*np.abs(x[0,0]-x[1,0])))
vel = .45

v[0,0]=vel
v[0,1]=vel

v[1,0]=-vel
v[1,1]=-vel

gridSize = np.array([5, 5, 5])

m=np.ones(n) #mass

dt_t = 0.5#1.0
parts=Particles(x,v,m, gridSize, grid_dx=dt_t)
colors = cm.rainbow(np.linspace(0, 1, n))

dt=0.01

# fig = plt.figure()
# ax = Axes3D(fig)
# ax.scatter(parts.x[n_i][0], parts.x[n_i][1], parts.x[n_i][2], color=colors[n_i])
# plt.pause(0.01)
# plt.show()

#looking at the density to see if it makes sense
"""
d = parts.get_density()

x=np.arange(0, gridSize[0], dt_t)
yy,zz=np.meshgrid(x,x)

for i in range(0, int(gridSize[0]/dt_t)):
    plt.pcolor(yy,zz, d[i],  shading='auto')
    plt.title("x = {}".format(i*dt_t))
    plt.xlabel("z")
    plt.ylabel("y")
    
    plt.pause(2)
plt.colorbar()
print(d)
"""

phi_test, f_test, grad_x, grad_y, grad_z = parts.get_forces_conv()
g = parts.G_laplacian()

# x=np.arange(0, gridSize[0], dt_t)
# xx,yy,zz=np.meshgrid(x,x,xs)

x_g = np.arange(0, gridSize[0], dt_t)
y_g = np.arange(0, gridSize[1], dt_t)
z_g = np.arange(0, gridSize[2], dt_t)
grid = np.meshgrid(x_g, y_g, z_g)
xx,yy,zz = np.meshgrid(x_g, y_g, z_g)


# for i in range(0, int(gridSize[0]/dt_t)):
#     plt.pcolor(yy,zz, f_test[i],  shading='auto')
#     plt.title("x = {}".format(i*dt_t))
#     plt.xlabel("z")
#     plt.ylabel("y")
    
#     plt.pause(2)
# plt.colorbar()

# print(grid[0].shape, grad_x.shape)

# print(grad_x)

# fig = plt.figure()
# ax = fig.gca(projection='3d')
# for n_i in range(n):
#     ax.scatter(parts.x[n_i][0], parts.x[n_i][1], parts.x[n_i][2], color=colors[n_i])
# ax.quiver(xx, yy, zz, grad_x, grad_y, grad_z)#, length=1.0)
# plt.pause(5)

x=np.arange(0, gridSize[0], dt_t)
yy,zz=np.meshgrid(x,x)

# for i in range(0, int(gridSize[0]/dt_t)):
#     plt.pcolor(yy,zz, phi_test[i],  shading='auto')
# #     plt.pcolor(yy,zz, g[i],  shading='auto')
    
#     plt.title("x = {}".format(i*dt_t))
#     plt.xlabel("z")
#     plt.ylabel("y")
    
#     plt.pause(2)
# plt.colorbar()

# """
colors = cm.rainbow(np.linspace(0, 1, n)) 

fig = plt.figure()
ax = Axes3D(fig)
ax.set_xlabel('x')
ax.set_ylabel('y')
ax.set_zlabel('z')
        
for k in range(300):
    for l in range(5):
        parts.take_step_leapfrog_conv(dt)
    
    print('step ',k)

    #     plt.clf() #clears prev graphs
    for n_i in range(n):
        if k==0:
            ax.scatter(parts.x[n_i][0], parts.x[n_i][1], parts.x[n_i][2])#, color=colors[n_i])
        else:
            ax.scatter(parts.x[n_i][0], parts.x[n_i][1], parts.x[n_i][2], color=colors[n_i])
    ax.set_title("frame {}".format(k),fontsize=20)
    plt.savefig('twoPart_{}.jpg'.format(k), dpi=1200)
    print("E total:", parts.get_energy())

#     plt.pause(0.01)
# """   
l=8

E total: [5.915082110156451, 0.10643751478932151, 5.808644595367129]
step  106
E total: [5.882790161623229, 0.07414556625610036, 5.808644595367129]
step  107
E total: [5.858608328903526, 0.049963733536397076, 5.808644595367129]
step  108
E total: [5.841020908036543, 0.0323763126694144, 5.808644595367129]
step  109
E total: [5.828673167527912, 0.02002857216078244, 5.808644595367129]
step  110
E total: [5.8203713483496875, 0.01172675298255866, 5.808644595367129]
step  111
E total: [5.815082663940357, 0.00643806857322784, 5.808644595367129]
step  112
E total: [5.811935300204831, 0.0032907048377020997, 5.808644595367129]
step  113
E total: [5.81021841551445, 0.001573820147320906, 5.808644595367129]
step  114
E total: [5.809382140706981, 0.000737545339851052, 5.808644595367129]
step  115
E total: [5.809037579086616, 0.0003929837194866677, 5.808644595367129]
step  116
E total: [5.8089568064239785, 0.00031221105684921876, 5.808644595367129]
step  117
E total: [5.809072870956117, 0.00042827558

KeyboardInterrupt: 

### Part 3: 
Set up both periodic and non-periodic boundary conditions. Set up a problem where hundreds of thousands of particles are initially scattered randomly throughout the domain. Show the evolution with time for both periodic and non-periodic boundary conditions. Track the total energy how well is it conserved? 

In [ ]:
n=500000 #runs really quite slow with even just 10 000 points
gridSize = [15, 15, 15]

#single particle at rest
x=np.random.uniform(0, gridSize[0],(n,3))
v=np.random.randn(n,3)*0 #at rest?

# print(x)

m=np.ones(n) #mass

dt_t = 1.

# parts=Particles(x,v,m, gridSize, grid_dx=dt_t)
parts=Particles(x,v,m, gridSize, grid_dx=dt_t, BC_per=True)

dt=0.01

# https://colab.research.google.com/drive/1zc1nvra1HF01mTtpOlRigj-UBVzYgFqq#scrollTo=pvtJPFS3LfYv

fig = plt.figure()
ax=fig.add_subplot(111,projection="3d")
# ax = Axes3D(fig)
ax.set_xlabel("x",fontsize=14)
ax.set_ylabel("y",fontsize=14)
ax.set_zlabel("z",fontsize=14)

ax.set_xlim3d(0, gridSize[0])
ax.set_ylim3d(0,gridSize[1])
ax.set_zlim3d(0,gridSize[2])

# ax.set_title("Initial conditions of the Universe\n",fontsize=20)
# scatt = ax.scatter(parts.x[:,0], parts.x[:,1], parts.x[:,2], color="royalblue",marker=".")#,s=.1)
# plt.pause(.4)

save_E = []

for k in range(40):
    for l in range(1):
        parts.take_step_leapfrog_conv(dt)
   
    scatt.remove() #clears prev points
    scatt = ax.scatter(parts.x[:,0], parts.x[:,1], parts.x[:,2], color="royalblue",marker=".", s=0.003)#alpha=0.7, )
    
    ax.set_title("frame {}".format(k),fontsize=20)
    plt.savefig('3dfinal_P_{}.jpg'.format(k), dpi=1200)
#     ax.set_title("frame {}".format(k))
#     ax.text2D(0.05, 0.95, "frame {}".format(k), transform=ax.transAxes)

    save_E.append(parts.get_energy())
    print("Step: {}, E total: {}".format(k, save_E[-1][0]))
    

In [ ]:
save_E = np.array(save_E)
steps = np.linspace(0, len(save_E[:,0])-1, len(save_E[:,0]))

fig = plt.figure()
plt.plot(steps, save_E[:,0], label="Total E")
plt.plot(steps, save_E[:,1], label="Kinetic E", ls="--") 
plt.plot(steps, save_E[:,2], label="Potential E") #hmm this is constant - should it be?

plt.title("Energy over time")
plt.xlabel("time step")
plt.ylabel("Energy")
plt.legend()

plt.pause(5.)

In [ ]:
n=100000 #runs really quite slow with even just 10 000 points
gridSize = [15, 15, 15]

#single particle at rest
x=np.random.uniform(1.0, gridSize[0]-1.0,(n,3)) #need to place closer to center
v=np.random.randn(n,3)*0 #at rest?

# print(x)

m=np.ones(n)*5 #mass

dt_t = 1.

# parts=Particles(x,v,m, gridSize, grid_dx=dt_t)
parts=Particles(x,v,m, gridSize, grid_dx=dt_t, BC_per=False)

dt=0.01

# https://colab.research.google.com/drive/1zc1nvra1HF01mTtpOlRigj-UBVzYgFqq#scrollTo=pvtJPFS3LfYv

fig = plt.figure()
ax=fig.add_subplot(111,projection="3d")
# ax = Axes3D(fig)
ax.set_xlabel("x",fontsize=14)
ax.set_ylabel("y",fontsize=14)
ax.set_zlabel("z",fontsize=14)

ax.set_xlim3d(0, gridSize[0])
ax.set_ylim3d(0,gridSize[1])
ax.set_zlim3d(0,gridSize[2])

# ax.set_title("Initial conditions of the Universe\n",fontsize=20)
# scatt = ax.scatter(parts.x[:,0], parts.x[:,1], parts.x[:,2], color="royalblue",marker=".")#,s=.1)
# plt.pause(.4)

save_E_w = []

for k in range(40):
    for l in range(1):
        parts.take_step_leapfrog_conv(dt)
   
    scatt.remove() #clears prev points
    scatt = ax.scatter(parts.x[:,0], parts.x[:,1], parts.x[:,2], color="royalblue",marker=".", s=0.08)#alpha=0.7, )
    
    ax.set_title("frame {}".format(k),fontsize=20)
    plt.savefig('3dfinal_wrap_{}.jpg'.format(k), dpi=1200)
#     ax.set_title("frame {}".format(k))
#     ax.text2D(0.05, 0.95, "frame {}".format(k), transform=ax.transAxes)

    save_E_w.append(parts.get_energy())
    print("Step: {}, E total: {}".format(k, save_E[-1][0]))

### Part 4: 
In cosmology, we start the universe with a scale-invariant power spectrum, so mass fluctuations are proportional to $k^{−3}$. Start with the particles on a grid, but with masses derived from a realization of $k^{−3}$ and use periodic boundary conditions (although not strictly necessary, you may with to start with your particles in the center of grid cells rather than at the corners). How does your universe look now?

In [ ]:
"""
what is k?? - wavenumber - but of what?

you create a 3d grid with a k^3 dependence, apply noise, and then apply an inverse fourier transform. 
This gives you a density map with a k^3 spatial dependence
"""

def r_grid(grid, dt, point):
    t = np.zeros( (len(grid[0]), len(grid[1]), len(grid[2])) ) 
    grid_arr = np.array(grid)
    t = np.sum((grid_arr + (dt)/2.0 -point[0])**2,axis=0)

    return t

gridSize = [100, 100, 100]
grid_dx = 1.0

x_g = np.arange(0, gridSize[0], grid_dx)
y_g = np.arange(0, gridSize[1], grid_dx)
z_g = np.arange(0, gridSize[2], grid_dx)
grid = np.meshgrid(x_g, y_g, z_g)

# k_grid = r_grid(grid, dt, [0,0,0])
k_grid = np.ones(gridSize)*grid_dx

In [ ]:
# rgrid_arr

noise = np.random.normal(0,1,k_grid.shape)

k_grid_noise = k_grid + noise*0

kgrid_inv = np.fft.irfftn(k_grid_noise, s=k_grid_noise.shape)


In [ ]:
# print(k_grid)

x=np.arange(0, gridSize[0], grid_dx)
yy,zz=np.meshgrid(x,x)
# """
# for i in range(0, int(gridSize[0]/grid_dx)):
for i in range(10, 15):
    
    plt.pcolor(yy, zz, kgrid_inv[i],  shading='auto')
#     plt.pcolor(yy,zz, g[i],  shading='auto')
    
    plt.title("x = {}".format(i*dt_t))
    plt.xlabel("z")
    plt.ylabel("y")
    plt.colorbar()
    plt.pause(2)
    plt.clf()
# """